# Load a model and generate poems

This notebook has two objectives:
    - load a trained poem generator
    - load a trained rhyme generator
    - Generate some poetry
    

## Importing packages and loading models
We start by importing a couple of packages and load models previously trained

In [1]:
import tensorflow as tf  # version 1.9 or above
tf.enable_eager_execution()  # Execution of code as it runs in the notebook. Normally, TensorFlow looks up the whole code before execution for efficiency.
from tensorflow.keras.layers import Embedding, GRU, Dense
import numpy as np
import re
from tensorflow.train import AdamOptimizer
from tensorflow.losses import sparse_softmax_cross_entropy
import time

## Load models and hyperparameters

Now we can load hyperparameters for both the poem generator and the rhyme generator

Let's start with the poem generator

In [2]:
# Load hyperparameters and layers' weights previously saved
hyperparameters_poems = np.load('hyperparameters_poems.npy')[()]
embedding_weights_poems = np.load('embedding_weights_poems.npy')
gru_weights_poems = np.load('gru_weights_poems.npy')
fc_weights_poems = np.load('fc_weights_poems.npy')
char2idx_poems = np.load('char2idx_poems.npy')[()]
idx2char_poems = np.load('idx2char_poems.npy')[()]

# Hyperparameters
max_length_poems = hyperparameters_poems['max_length']  # Maximum length sentence we want per input in the network
embedding_dim_poems = hyperparameters_poems['embedding_dim']  # number of 'meaningful' features to learn. Ex: ['queen', 'king', 'man', 'woman'] has a least 2 embedding dimension: royalty and gender.
units_poems = hyperparameters_poems['units']  # In keras: number of output of a sequence. In short it rem
BATCH_SIZE_poems = hyperparameters_poems['BATCH_SIZE']
BUFFER_SIZE_poems = hyperparameters_poems['BUFFER_SIZE']
vocab_size_poems = len(dict(idx2char_poems))

Now let's load the hyperparameters and weights for the rhyme generator

In [3]:
# Load hyperparameters and layers' weights previously saved
hyperparameters_rhymes = np.load('hyperparameters_rhymes.npy')[()]
embedding_weights_rhymes = np.load('embedding_weights_rhymes.npy')
gru_weights_rhymes = np.load('gru_weights_rhymes.npy')
fc_weights_rhymes = np.load('fc_weights_rhymes.npy')
char2idx_rhymes = np.load('word2idx_rhymes.npy')[()]
idx2char_rhymes = np.load('idx2word_rhymes.npy')[()]

# Hyperparameters
max_length_rhymes = hyperparameters_rhymes['max_length']  # Maximum length sentence we want per input in the network
embedding_dim_rhymes = hyperparameters_rhymes['embedding_dim']  # number of 'meaningful' features to learn. Ex: ['queen', 'king', 'man', 'woman'] has a least 2 embedding dimension: royalty and gender.
units_rhymes = hyperparameters_rhymes['units']  # In keras: number of output of a sequence. In short it rem
BATCH_SIZE_rhymes = hyperparameters_rhymes['BATCH_SIZE']
BUFFER_SIZE_rhymes = hyperparameters_rhymes['BUFFER_SIZE']
vocab_size_rhymes = len(dict(idx2char_rhymes))

## Models creation

We now reproduce models with the same structure as the ones previously trained. 

*Note: There is no need for declaring two classes (one for proems, the other for rhymes). Indeed, both poems and rhymes models are based on the same architecture.*

In [4]:

class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size):
        super(Model, self).__init__()
        self.units = units
        self.batch_sz = batch_size
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = GRU(self.units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, states = self.gru(x, initial_state=hidden)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, states

    
model_poems = Model(vocab_size_poems, embedding_dim_poems, units_poems, BATCH_SIZE_poems)
model_rhymes = Model(vocab_size_rhymes, embedding_dim_rhymes, units_rhymes, BATCH_SIZE_rhymes)

However, we now face a problem when we want to change a layer's weights

Ideally, changing weights should fit in one line of code like:

```
model_poem.gru.set_weights(gru_weights)
```
But it throws an error. A dirty get around consists in retraining the model on a very small dataset

## Getting around weight initialization issues

*I am improving this part*

We simply retrain the model on one epoch and a small sample. We have to do it for both the poem generator and the rhyme generator.

First, the poem generator

In [5]:
# This is merely a copy of poem_model
path = 'corpus.txt'
with open(path, encoding='utf-8') as f:
    text = f.read().lower()
text = text[: 10 * BUFFER_SIZE_poems + 1]  # Not sure BUFFER_SIZE + 1 would always be enough

text = re.sub('[^a-z\n]', ' ', text)
text = text[::-1]  # Put the text backwards

input_text = []
target_text = []

for f in range(0, len(text) - max_length_poems, max_length_poems):
    inps = text[f : f + max_length_poems]
    targ = text[f + 1 : f + 1 + max_length_poems]
    input_text.append([char2idx_poems[i] for i in inps])
    target_text.append([char2idx_poems[t] for t in targ])
    
dataset = tf.data.Dataset.from_tensor_slices((input_text, target_text)).shuffle(BUFFER_SIZE_poems)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE_poems))


optimizer = AdamOptimizer()

def loss_function(real, preds):
    return sparse_softmax_cross_entropy(labels=real, logits=preds)


hidden = model_poems.reset_states()  # initializes the hidden state at the start of every epoch

for (batch, (inp, target)) in enumerate(dataset):
      with tf.GradientTape() as tape:
          predictions, hidden = model_poems(inp, hidden)  # feeds the hidden state back into the model
          target = tf.reshape(target, (-1, ))  # reshapes for the loss function
          loss = loss_function(target, predictions)

      grads = tape.gradient(loss, model_poems.variables)
      optimizer.apply_gradients(zip(grads, model_poems.variables), global_step=tf.train.get_or_create_global_step())

Then, we repeat the process with the rhyme generator

In [6]:
# This is merely a copy of rhyme_model
path = 'rhymes.txt'
with open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = text[: 100 * BUFFER_SIZE_rhymes]  # 100 * BUFFER_SIZE_rhymes may not always be enough
text = re.sub('[^a-z\n]', ' ', text)
text = text.split('\n')
print(text[:50])
input_text = []
target_text = []

for f in range(0, len(text) - max_length_rhymes, max_length_rhymes):
    inps = text[f : f + max_length_rhymes]
    targ = text[f + 1 : f + 1 + max_length_rhymes]
    input_text.append([char2idx_rhymes[i] for i in inps])
    target_text.append([char2idx_rhymes[t] for t in targ])
    
dataset = tf.data.Dataset.from_tensor_slices((input_text, target_text)).shuffle(BUFFER_SIZE_rhymes)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE_rhymes))


optimizer = AdamOptimizer()

def loss_function(real, preds):
    return sparse_softmax_cross_entropy(labels=real, logits=preds)
    
hidden = model_rhymes.reset_states()  # initializes the hidden state at the start of every epoch

for (batch, (inp, target)) in enumerate(dataset):
      with tf.GradientTape() as tape:
          predictions, hidden = model_rhymes(inp, hidden)  # feeds the hidden state back into the model
          target = tf.reshape(target, (-1, ))  # reshapes for the loss function
          loss = loss_function(target, predictions)

      grads = tape.gradient(loss, model_rhymes.variables)
      optimizer.apply_gradients(zip(grads, model_rhymes.variables), global_step=tf.train.get_or_create_global_step())

['men', 'be', 'sea', 'refrain', 'day', 'say', 'born', 'star', 'war', 'morn', 'birth', 'mirth', 'tree', 'men', 'again', 'thee', 'earth', 'birth', 'now', 'low', 'fell', 'so', 'vain', 'well', 'tree', 'fell', 'leaves', 'me', 'above', 'melody', 'tree', 'low', 'evermore', 'woe', 'land', 'glee', 'night', 'sea', 'land', 'night', 'appeared', 'light', 'lonely', 'day', 'brightness', 'away', 'raiment', 'night', 'darkness', 'light']


## Loading our weights back in the model

We can finally plug the weights we have loaded.

In [8]:
model_poems.embedding.set_weights(np.asarray(embedding_weights_poems))
model_poems.gru.set_weights(gru_weights_poems)
model_poems.fc.set_weights(fc_weights_poems)

model_rhymes.embedding.set_weights(np.asarray(embedding_weights_rhymes))
model_rhymes.gru.set_weights(gru_weights_rhymes)
model_rhymes.fc.set_weights(fc_weights_rhymes)

## Text Generation

Finally, we can generate some poetry text from our model.

First the rhymes:

In [7]:
temperature = 0.4

num_generate = 20  # number of characters to generate
start_string = ['fell', 'vain', 'well', 'tree', 'fell', 'leave', 'me', 'above', 'melody']  # beginning of the generated text. TODO: try start_string = ' '
input_eval = [char2idx_rhymes[s] for s in start_string]  # converts start_string to numbers the model understands
input_eval = tf.expand_dims(input_eval, 0)  # 

text_generated = []


hidden = [tf.zeros((1, units_rhymes))]
for i in range(num_generate):
    predictions, hidden = model_rhymes(input_eval, hidden)  # predictions holds the probabily for each character to be most adequate continuation
   
    predictions = predictions / temperature  # alters characters' probabilities to be picked (but keeps the order)
    predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()  # picks the next character for the generated text
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated += [idx2char_rhymes[predicted_id]]

rhymes = text_generated

In [ ]:
# Evaluation step(generating text using the model learned)


for temperature in [0.5, 0.7, 0.9, 1.1, 1.5]:
    print('Temperature = {} \n'.format(temperature))
    text_generated = ''
    for rhyme in rhymes:
    
        num_generate = 150  # number of characters to generate
        start_string = text_generated + rhyme[::-1]  # beginning of the generated text. TODO: try start_string = ' '
        input_eval = [char2idx_poems[s] for s in start_string]  # converts start_string to numbers the model understands
        input_eval = tf.expand_dims(input_eval, 0)  # 
        hidden = [tf.zeros((1, units_poems))]
        
        b = True
        c = 1
        addition = ' '
        tmp = start_string + ' '
        while b == True:
            predictions, hidden = model_poems(input_eval, hidden)  # predictions holds the probabily for each character to be most adequate continuation
           
            predictions = predictions / temperature  # alters characters' probabilities to be picked (but keeps the order)
            predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()  # picks the next character for the generated text
            input_eval = tf.expand_dims([predicted_id], 0)
            tmp += idx2char_poems[predicted_id]
            addition += idx2char_poems[predicted_id]
            c += 1
            if idx2char_poems[predicted_id] == '\n' or c > num_generate:
                text_generated += rhyme[::-1] + addition
                b = False
    print('Poem : \n')
    print (text_generated[::-1])

Temperature = 0.5 

Poem : 


then crossed the little fern and blooms her grandme
in a golden hair andme
and she answered   i have droundme
there in silence andme
and listened  and smiled  and she was cuddme
the wind is in the shrandme
that s the way for liberty  its pictures drandme
he lived in the rose andme
and the glory of gladness and its suddme
for he loves the singled hair andme
an  then he asked for andme
it is a place in the silentless andme
and say that they ever promessions andme
it seemed to thinking in the andme
and he was in the other in the golden hair andme
crushed in silver grandme
and rider fair andme
and said   never every one  as she cuddme
where the visions andme
that which was the vision and gladme
Temperature = 0.7 

